# Double Integrator Model

$$\dot{q} = u$$ 
$$ \dot{v}_i = u_i \text{  } i=1,\dots,n$$
where $q_i$ is postion and $v_i$ is velocity. 

$$\dot{e}_{ij} = \frac{\tilde{q}_{ij}^\top(v_i-v_j)}{e_{ij}+d_{ij}}$$
 $$\dot{W} = \frac{1}{2} z^\top \dot{z} = z^\top R(\tilde{q})v$$ 
 
 Using backstepping tecqnique to change the Lyapunov function. Let
$$s=v-v_f$$

where $v_f$ is fictitious (or desired) velocity input, which will
be specified later. The variable s quantifies the error between the actual agent velocity and the desired velocity-level input.


$$W_d(e,s) = W(e) + \frac{1}{2}s^\top s$$ 

is the augmented Lyapunov function candidate,
$$\dot{W} = z^\top R(\tilde{q})v + s^\top \dot{s}$$ 
$$ = z^\top R(\tilde{q})(s+v_f) + s^\top (u-\dot{v_f})$$



In [21]:

using DifferentialEquations, Plots; pyplot();
using LinearAlgebra 
theme(:ggplot2)

In [22]:
n = 6;
kv = 0.05;
ka = 0.1;
Adj = [0 1 0 0 0 1;
       1 0 1 0 0 1;
       0 1 0 1 0 1;
       0 0 1 0 1 1;
       0 0 0 1 0 1;
       1 1 1 1 1 0];

d = zeros(n,n); 
c1 = cos(2*pi/5);
c2 = cos(pi/5);
s1 = sin(2*pi/5);
s2 = sin(4*pi/5);
x_coor = [0; -s1; -s2; s2; s1; 0];  
y_coor = [1; c1; -c2; -c2; c1; 0];  
z_coor = [0.0; 0.0; 0.0; 0.0; 0.0; 0.0];

for ii = 1:n
    for jj = 1:n
        d[ii,jj] = sqrt((x_coor[ii]-x_coor[jj])^2+(y_coor[ii]-y_coor[jj])^2+(z_coor[ii]-z_coor[jj])^2)
    end
end


ub = 0.5;                           # Upper bound for random ini. condition
lb = -0.5;                          # Lower bound for random ini. condition
tfinal = 4;


In [23]:
mutable struct para2
n
kv
ka
Adj
d
end

In [24]:
p2 = para2(n,kv,ka,Adj,d)

para2(6, 0.05, 0.1, [0 1 … 0 1; 1 0 … 0 1; … ; 0 0 … 0 1; 1 1 … 1 0], [0.0 1.1755705045849463 … 1.1755705045849463 1.0; 1.1755705045849463 0.0 … 1.902113032590307 0.9999999999999999; … ; 1.1755705045849463 1.902113032590307 … 0.0 0.9999999999999999; 1.0 0.9999999999999999 … 0.9999999999999999 0.0])

In [30]:
q_0 = [x_coor y_coor z_coor]'+(lb*ones(3,n)+(ub-lb)*rand(3,n))
v_0 = (lb*rand(3,n)+(ub-lb)*rand(3,n))*2
qv_0 = [q_0 v_0];
qv_0_vec = reshape(qv_0,(1,:));
time_span = (0.0,4.0);

In [32]:
function DI_formation(ddu,u,p,t)
    n = p.n
    kv = p.kv
    ka = p.ka
    Adj = p.Adj
    d = p.d
    
    qv = reshape(u,(3,:))
    q = qv[:,1:n];
    v = qv[:,n+1:2*n];
    
    z = zeros(3*n-6,1);
    R = zeros(3*n-6,3*n);
    R_dot  = zeros(3*n-6,3*n);
    e = zeros(n,n);
    
    ord = 1
    for i=1:n-1
        for j=i+1:n
            e[i,j] = sqrt((q[:,i]-q[:,j])'*(q[:,i]-q[:,j]))-d[i,j]
            # print("i =",i,"j=",j,"\n")
            if Adj[i,j] == 1
                z[ord] = e[i,j]*(e[i,j]+2*d[i,j]);
                R[ord,3*i-2:3*i] = (q[:,i]-q[:,j])';
                R[ord,3*j-2:3*j] = (q[:,j]-q[:,i])';
                R_dot[ord,3*i-2:3*i] = (v[:,i] - v[:,j])';
                R_dot[ord,3*j-2:3*j] = (v[:,j] - v[:,i])';
                ord = ord+1;
            end
        end
    end
    
    v = u[3*n+1:6*n];
    ua = -kv*R'*z;
    vf = ua;
    vfdot = -kv*R_dot'*z-kv*R'*2*R*v;
    s = v - vf;
    control = -ka*s + vfdot -R'*z;
    ddu .= control


end


DI_formation (generic function with 1 method)

In [33]:
prob3 = ODEProblem(DI_formation,qv_0_vec,time_span,p2)

sol3 = solve(prob3,saveat=0.1)

pos3 = sol3.u

DimensionMismatch: DimensionMismatch("array could not be broadcast to match destination")